# Modis-tempCNN mapping model
Train and save the Modis-tempCNN model for map generation 

In [1]:
import os
import json
import numpy as np
import pandas as pd

import initialise
import common
from model_utils import reshape_data
from modelling_functions import create_models
from architecture_modis_tempCNN import model_params

## Input files
Change these settings as required
- `modis_csv`: The file containing extracted MODIS data for each sample, created by `Extract MODIS Data.ipynb`
- `aux_csv`: The file containing extracted DEM and other auxiliary data for each sample, created by `Extract DEM Data.ipynb`. This includes the labels and pre-processed site location and sampling date data from Globe-LFMC

In [2]:
modis_csv = os.path.join(common.DATASETS_DIR, 'modis_365days.csv')
aux_csv = os.path.join(common.DATASETS_DIR, 'samples_365days.csv')

## Set up model parameters

### Model parameters settings
To find out more about any parameter, run `model_params.help('<parameter>')`. 

In [3]:
model_params['modelName'] = 'modis-tempCNN_map2017'
model_params['description'] = 'Build model for 2017 maps'
model_params['modisFilename'] = modis_csv
model_params['auxFilename'] = aux_csv
model_params['saveModels'] = [common.MODIS_TEMPCNN_MODEL]   # Save the model to use for map-wide estimates
model_params['splitMethod'] = None                          # Train using all the data
model_params['seedList'] = [441]
model_params['tempDir'] = common.TEMP_DIR
model_params['modelDir'] = os.path.join(common.MODELS_DIR, model_params['modelName'])

restart = False     # Change to True if retrying/restarting this script
if not os.path.exists(model_params['modelDir']):
    os.makedirs(model_params['modelDir'])
elif not restart:   # Don't over-write something important!
    raise FileExistsError(f"{model_params['modelDir']} exists but restart not requested")

model_params

{'modelName': 'modis-tempCNN_map2017',
 'description': 'Build model for 2017 maps',
 'modelClass': 'LfmcTempCnn',
 'modelDir': 'G:\\My Drive\\LFMC Data\\multi_modal_LFMC\\Models\\modis-tempCNN_map2017',
 'tempDir': 'C:\\Temp\\LFMC',
 'diagnostics': False,
 'dataSources': ['modis', 'aux'],
 'restartRun': None,
 'derivedModels': {'merge10': {'type': 'merge', 'models': 10}},
 'saveModels': ['merge10'],
 'saveTrain': None,
 'saveValidation': True,
 'plotModel': True,
 'randomSeed': 1234,
 'modelSeed': 1234,
 'modelRuns': 1,
 'resplit': False,
 'seedList': [441],
 'maxWorkers': 1,
 'deterministic': False,
 'gpuDevice': 0,
 'gpuMemory': 0,
 'modisFilename': 'G:\\My Drive\\LFMC Data\\multi_modal_LFMC\\Datasets\\modis_365days.csv',
 'modisChannels': 7,
 'modisNormalise': {'method': 'minMax', 'percentiles': 2},
 'prismFilename': None,
 'prismChannels': 7,
 'prismNormalise': {'method': 'minMax', 'percentiles': 2},
 'auxFilename': 'G:\\My Drive\\LFMC Data\\multi_modal_LFMC\\Datasets\\samples_365d

## Prepare the data

In [4]:
modis_data = pd.read_csv(model_params['modisFilename'], index_col=0)
x_modis = reshape_data(np.array(modis_data), model_params['modisChannels'])
print(f'Modis shape: {x_modis.shape}')

aux_data = pd.read_csv(model_params['auxFilename'], index_col=0)
y = aux_data[model_params['targetColumn']]

Modis shape: (66946, 365, 7)


## Build and run the model
Builds and trains the LFMC model. After training the model, several derived models are created and evaluated. The full list of derived models is:
- `base` - The fully trained model
- `merge10` - A model created by merging the last 10 checkpoints. The checkpoints are merged by averaging the corresponding weights from each model.

The model and training statistics are saved to `model_dir`.

In [5]:
X = {'modis': x_modis}
with open(os.path.join(model_params['modelDir'], 'model_params.json'), 'w') as f:
    model_params.save(f)
model = create_models(model_params, aux_data, X, y)

Auxiliary columns: ['Day_sin', 'Day_cos', 'Long_sin', 'Long_cos', 'Lat_norm', 'Elevation', 'Slope', 'Aspect_sin', 'Aspect_cos']
modis shape: (66946, 365, 7)
aux shape: (66946, 9)
